In [12]:
import pandas as pd
combined_dataset = pd.read_pickle('../combined_dataset_notes.pkl')


combined_dataset = combined_dataset.drop(['HADM_ID'], axis=1)
print(combined_dataset.shape)
#combined_dataset.head()

only_text = combined_dataset[["text","G1","G2","G3","G4","G5","G6","G7","G8","G9","G10","G11","G12","G13","G14","G15","G16","G17","G18","G19","G20"]]
#print(only_text.shape)

!pip install --upgrade gensim
!pip install -U gensim


from gensim.models import Word2Vec

new_model = Word2Vec.load('model750-5-5.bin')
#print(new_model)

from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.model_selection import GridSearchCV,StratifiedKFold,KFold
from sklearn.metrics import jaccard_score,roc_auc_score,confusion_matrix,hamming_loss


(45526, 504)
Requirement already up-to-date: gensim in /home/akshara/anaconda3/lib/python3.7/site-packages (3.8.1)
Requirement already up-to-date: gensim in /home/akshara/anaconda3/lib/python3.7/site-packages (3.8.1)


In [13]:
combined_dataset.head()

,text,ADMISSION_DAYS,GENDER,AGE,50801,50802,50803,50804,50805,50806,...,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20
47366,"[chest, pa, lat, clip, clip, number, radiology...",7,0,35,462.0,0.0,24.0,26.0,1.0,104.0,...,0,1,0,0,0,0,0,0,0,1
47024,"[pm, liver, gallbladder, u, single, organ, cli...",5,1,60,462.0,-6.0,24.0,17.0,1.0,104.0,...,0,1,0,0,0,1,0,0,0,0
9046,"[chest, portable, ap, clip, clip, number, radi...",13,0,49,462.0,0.0,24.0,26.0,1.0,104.0,...,0,0,0,0,0,1,0,0,0,1
843,"[pm, chest, pa, lat, clip, clip, number, radio...",5,1,61,462.0,2.0,24.0,28.0,1.0,103.0,...,0,0,0,0,0,0,0,1,0,1
43466,"[pm, chest, portable, ap, different, physician...",5,0,54,462.0,0.0,23.0,25.0,1.0,106.0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
y = only_text.iloc[:,1:]
x = only_text.iloc[:,:1]

print(x.shape,y.shape)

(45526, 1) (45526, 20)


In [15]:
x_full = combined_dataset.iloc[:,:484]
y_full = combined_dataset.iloc[:,484:]
print(x_full.shape,y_full.shape,x_full.columns,y_full.columns)

(45526, 484) (45526, 20) Index([          'text', 'ADMISSION_DAYS',         'GENDER',            'AGE',
                  50801,            50802,            50803,            50804,
                  50805,            50806,
       ...
                  51512,            51513,            51514,            51515,
                  51516,            51517,            51518,            51519,
                  51529,            51532],
      dtype='object', length=484) Index(['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9', 'G10', 'G11',
       'G12', 'G13', 'G14', 'G15', 'G16', 'G17', 'G18', 'G19', 'G20'],
      dtype='object')


In [16]:
import numpy as np
WordVectorz=dict(zip(new_model.wv.index2word,new_model.wv.vectors))
class AverageEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim =100 # because we use 100 embedding points 

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
AverageEmbeddingVectorizer(WordVectorz).fit(x,y)
x.text = AverageEmbeddingVectorizer(WordVectorz).transform(x.text)






/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [17]:
x_full['text'] = x.text
x_full

,text,ADMISSION_DAYS,GENDER,AGE,50801,50802,50803,50804,50805,50806,...,51512,51513,51514,51515,51516,51517,51518,51519,51529,51532
47366,0.421995,7,0,35,462.0,0.0,24.0,26.0,1.0,104.0,...,1.0,1.044,1.0,1.0,2.0,3.0,0.0,0.0,135.5,43.0
47024,0.102772,5,1,60,462.0,-6.0,24.0,17.0,1.0,104.0,...,1.0,1.044,2.0,1.0,7.0,3.0,0.0,0.0,135.5,43.0
9046,0.241119,13,0,49,462.0,0.0,24.0,26.0,1.0,104.0,...,1.0,1.044,1.0,1.0,2.0,3.0,0.0,0.0,135.5,43.0
843,0.126102,5,1,61,462.0,2.0,24.0,28.0,1.0,103.0,...,1.0,1.044,1.0,1.0,1.0,3.0,0.0,0.0,135.5,43.0
43466,0.348608,5,0,54,462.0,0.0,23.0,25.0,1.0,106.0,...,1.0,1.044,1.0,1.0,2.0,3.0,0.0,0.0,135.5,43.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24035,0.371063,25,1,60,653.0,1.0,26.0,28.0,1.3,108.0,...,1.0,1.044,1.0,1.0,2.0,3.0,0.0,0.0,135.5,43.0
30073,0.451681,10,0,58,343.0,-3.0,24.0,28.0,1.0,104.0,...,1.0,1.044,1.0,1.0,2.0,3.0,0.0,0.0,135.5,43.0
25219,-0.169312,17,1,23,240.0,4.0,24.0,29.0,1.0,104.0,...,1.0,1.044,1.0,1.0,2.0,3.0,0.0,0.0,135.5,43.0
33394,-0.105971,6,1,70,462.0,2.0,24.0,26.0,1.0,105.0,...,1.0,1.044,4.0,1.0,2.0,3.0,0.0,0.0,135.5,43.0


In [18]:
from sklearn.model_selection import train_test_split
seed=243 
x_train_full,x_test_full,y_train_full,y_test_full = train_test_split(x_full,y_full,test_size=0.15,random_state = seed)
print(x_train_full.shape,y_train_full.shape,y_test_full.shape)

(38697, 484) (38697, 20) (6829, 20)


In [19]:
lgb_full = OneVsRestClassifier(LGBMClassifier(iterations=1000,learning_rate=0.15,n_jobs=-1))
lgb_full.fit(x_train_full,y_train_full)

OneVsRestClassifier(estimator=LGBMClassifier(boosting_type='gbdt',
                                             class_weight=None,
                                             colsample_bytree=1.0,
                                             importance_type='split',
                                             iterations=1000,
                                             learning_rate=0.15, max_depth=-1,
                                             min_child_samples=20,
                                             min_child_weight=0.001,
                                             min_split_gain=0.0,
                                             n_estimators=100, n_jobs=-1,
                                             num_leaves=31, objective=None,
                                             random_state=None, reg_alpha=0.0,
                                             reg_lambda=0.0, silent=True,
                                             subsample=1.0,
                           

In [28]:
from sklearn.model_selection import train_test_split
seed=243 
x_train_text,x_test_text,y_train_text,y_test_text = train_test_split(x,y,test_size=0.15,random_state = seed)
print(x_train_text.shape,y_train_text.shape,y_test_text.shape)

(38697, 1) (38697, 20) (6829, 20)


In [8]:
''' for str 
from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler(feature_range=(0, 1))
x_train_mm = mm.fit_transform(x_train_full)
x_test_mm = mm.transform(x_test_full)#needed here,i think u might need it for text also....'''

In [ ]:
'''
lgbm_text = joblib.load('w2v_lgbm.pkl')'''

In [9]:
'''weights = {0:0.28,1:0.72}
svc = OneVsRestClassifier(SVC(class_weight=weights))
lr = OneVsRestClassifier(LogisticRegression(class_weight=weights))
# u take over lol what u want me to do? train LR with some nice params'''

In [ ]:
'''lr.fit(x_train_full,y_train_full)
svc.fit(x_train_full,y_train_full)
lr_str_pred = lr.predict(x_test_full)
svc_str_pred = svc.predict(x_test_full)'''

/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432:

In [20]:
# getting lgb proba on text and lr on str
y_pred_prob = lgb_full.predict_proba(x_test_full)
y_pred = lgb_full.predict(x_test_full)

In [21]:
print(jaccard_score(y_test_full,y_pred,average='micro'))
print(jaccard_score(y_test_full,y_pred,average='macro'))
print(roc_auc_score(y_test_full,y_pred))
print(hamming_loss(y_test_full,y_pred))

0.5008530196897015
0.38313090786641574
0.665623807042951
0.1820837604334456


In [22]:
from sklearn.externals import joblib  #0.66 0.18
lgbm_text = joblib.load('w2v_lgbm.pkl')

/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [37]:
lgb_pred_text_prob = lgbm_text.predict_proba(x_test_text.text)

TypeError: 'float' object is not iterable

In [36]:
x_test_text.text

21738    0.031789
47854   -0.296812
3615    -0.086334
37273    0.066688
12132   -0.334977
           ...   
5542     0.136934
57534    0.123680
40358    0.221546
24988    0.157372
28595    0.519328
Name: text, Length: 6829, dtype: float32

In [32]:
import numpy as np
def corr(a, b):
    counts = 0
  #first_df = pd.read_csv(first_file,index_col=0)
  #second_df = pd.read_csv(second_file,index_col=0)
  # assuming first column is `prediction_id` and second column is `prediction`
    for i in range(a.shape[0]):  
      #prediction = a[:,i] #first_df.columns[0]
      # correlation
        #print("Finding correlation row : {}".format(i))
      #print("Column to be measured: {}".format(prediction))
        cor = np.corrcoef(a[i,:],b[i,:])
        #print("Pearson's correlation score: {}".format(cor))
        if(cor[0][1] < 0.7):
            counts+=1
      #print("Kendall's correlation score: {}".format(first_df[prediction].corr(second_df[prediction],method='kendall')))
      #print("Spearman's correlation score: {}".format(first_df[prediction].corr(second_df[prediction],method='spearman')))
    return counts

In [33]:
l = [corr(y_pred_prob,lgb_pred_text_prob)]
l# this was high thats y its working better ig
#what was high? like correlation was low for many inspite o
# 4717 cat v cat
# 4909 lgbm vs cat

IndexError: index 1 is out of bounds for axis 0 with size 1